In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import os
import zipfile
import gdown

# Verificar si la carpeta existe
if not os.path.exists('db_speech_enhancement'):
    # Descargar archivo desde Google Drive
    print("La carpeta no existe.")
    url = 'https://drive.google.com/file/d/1lFks-4bBf_lgD8j4Y-8xG5HpmhQehs4d/view?usp=sharing'
    nombre_zip = 'db.zip'
    
    gdown.download(url, nombre_zip, quiet=True,fuzzy=True)
    
    # Descomprimir archivo zip
    with zipfile.ZipFile(nombre_zip, 'r') as zip_ref:
        zip_ref.extractall('.')
    
    # Eliminar el archivo zip
    os.remove(nombre_zip)
    print("Datos Descargados")
else:
    print("La carpeta ya existe.")


La carpeta ya existe.


In [5]:
db_labels = pd.read_csv("db_speech_enhancement\labels.csv")
db_labels.head()

,audio_name,partition,snr
0,af01_018.wav,train,9.769693
1,cf11_096.wav,train,9.363799
2,pm24_005.wav,train,2.519448
3,pf40_068.wav,train,3.437852
4,nf35_010.wav,train,4.976195


In [66]:
#Objetivo funcion: dado un path a un audio, genera como slaida un espectrograma guardado en una matriz numpy
import librosa
import numpy as np

def audio_to_espectro(path_audio):
    # Cargar el archivo de audio
    y, sr = librosa.load(path_audio)
    espectrograma = librosa.stft(y)
    magnitud = np.abs(espectrograma) 
    fase = np.angle(espectrograma)
    return np.stack((magnitud, fase), axis=-1)

def espectro_to_audio(matriz):
    magnitud = matriz[:,:,0]
    fase = matriz[:,:,1]
    parte_real = magnitud * np.cos(fase)
    parte_imaginaria = magnitud * np.sin(fase)
    return librosa.istft(parte_real + 1j * parte_imaginaria)


In [75]:
train_audio_name = db_labels[db_labels['partition'] == 'train']['audio_name']
test_audio_name = db_labels[db_labels['partition'] == 'test']['audio_name'] 
validation_audio_name = db_labels[db_labels['partition'] == 'validation']['audio_name']

In [96]:
n = len(train_audio_name)
train_espect_norm = np.empty((n,1025, 199, 2),dtype=np.float32)
for i in range(n):
    train_espect_norm[i] = audio_to_espectro(f"db_speech_enhancement/normalized_db/{train_audio_name[i].split('_')[0]}/{train_audio_name[i]}")

0
1


ValueError: could not broadcast input array from shape (1025,181,2) into shape (1025,199,2)

In [91]:
print(f"db_speech_enhancement/normalized_db/{train_audio_name[0].split('_')[0]}/{train_audio_name[0]}")
print(f"db_speech_enhancement/noise_db/{train_audio_name[0].split('_')[0]}/noisy_{train_audio_name[0]}")

db_speech_enhancement/normalized_db/af01/af01_018.wav
db_speech_enhancement/noise_db/af01/noisy_af01_018.wav


In [94]:
(audio_to_espectro(f"db_speech_enhancement/normalized_db/{train_audio_name[0].split('_')[0]}/{train_audio_name[0]}"))


array([[[ 1.1116228e+01,  0.0000000e+00],
        [ 2.0379370e+01,  0.0000000e+00],
        [ 2.2310570e+01,  0.0000000e+00],
        ...,
        [ 2.1234865e+01,  0.0000000e+00],
        [ 2.0565031e+01,  0.0000000e+00],
        [ 1.2756441e+01,  0.0000000e+00]],

       [[ 8.9895382e+00,  2.2170527e+00],
        [ 1.2322159e+01,  2.9919727e+00],
        [ 1.0965534e+01, -3.1264358e+00],
        ...,
        [ 1.0766696e+01,  3.1383367e+00],
        [ 1.1530623e+01, -3.1001511e+00],
        [ 1.0213426e+01, -2.3749857e+00]],

       [[ 4.6482668e+00, -1.6169069e+00],
        [ 1.8479840e+00,  5.5510473e-01],
        [ 4.4608751e-01,  2.6775408e+00],
        ...,
        [ 7.2825527e-01,  7.2740096e-01],
        [ 1.9803572e+00, -1.4900101e+00],
        [ 5.3151383e+00,  1.2682636e+00]],

       ...,

       [[ 7.6389918e-03, -5.5151223e-03],
        [ 3.8089000e-03,  3.1360497e+00],
        [ 1.6075967e-07, -2.1138394e+00],
        ...,
        [ 1.8720216e-07, -7.3724455e-01],
     

In [ ]:
#objetivo generar un arreglo de espectorgramas dado un conjunto:
